In [4]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

**Load The Data**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math

In [7]:
import random
import numpy as np
from numpy import ndarray

In [8]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 9 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [9]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/CO2_QM_ds_mod_augmented.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [10]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
#LQ = 500       # Large Query: 500 queries
LQ = 1_000     # Large Query: 1K queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, LQ)
]

In [17]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


**-----------------Ball Tree--------------------------**

In [12]:
import numpy as np
from collections import deque

#*-----------------------------------------------------------------------------
class BallNode:
    def __init__(self):
        self.center = None
        self.radius = None
        self.left = None
        self.right = None
        self.points = None
        self.pivot1 = None
        self.pivot2 = None

#*-----------------------------------------------------------------------------
class BallTree:
    def __init__(self, leaf_size=220):
        self.leaf_size = leaf_size
        self.root = None

    #*-----------------------------------------------------------------------------
    def _find_furthest_points(self, points):
        # First find point furthest from origin as first pivot
        distances = np.sum(points ** 2, axis=1)
        p1_idx = np.argmax(distances)
        pivot1 = points[p1_idx]

        # Find point furthest from first pivot
        distances = np.sum((points - pivot1) ** 2, axis=1)
        p2_idx = np.argmax(distances)
        pivot2 = points[p2_idx]

        return pivot1, pivot2

    #*-----------------------------------------------------------------------------
    def _compute_distances(self, points, pivot):
        return np.sqrt(np.sum((points - pivot) ** 2, axis=1))

    #*-----------------------------------------------------------------------------
    def build(self, points):
        if points is None or len(points) == 0:
            return None

        points = np.array(points)
        self.root = self._build_recursive(points)
        return self.root

    #*-----------------------------------------------------------------------------
    def _build_recursive(self, points):
        if len(points) <= self.leaf_size:
            node = BallNode()
            node.points = points
            node.center = np.mean(points, axis=0)
            if len(points) > 1:
                node.radius = np.max(self._compute_distances(points, node.center))
            else:
                node.radius = 0
            return node

        node = BallNode()

        # Find two furthest points as pivots
        pivot1, pivot2 = self._find_furthest_points(points)
        node.pivot1 = pivot1
        node.pivot2 = pivot2

        # Assign points to closest pivot
        dist_to_p1 = self._compute_distances(points, pivot1)
        dist_to_p2 = self._compute_distances(points, pivot2)

        left_mask = dist_to_p1 <= dist_to_p2
        left_points = points[left_mask]
        right_points = points[~left_mask]

        # Compute node properties
        node.center = np.mean(points, axis=0)
        node.radius = np.max(self._compute_distances(points, node.center))

        # Recursively build subtrees
        node.left = self._build_recursive(left_points)
        node.right = self._build_recursive(right_points)

        return node

    #*-----------------------------------------------------------------------------
    def search(self, point):
        if self.root is None:
            return float('inf'), None

        point = np.array(point)
        best_dist = float('inf')
        best_point = None

        # Use a priority queue for best-first search
        # Each entry is (lower_bound_distance, node)
        queue = deque([(0, self.root)])

        while queue:
            _, node = queue.popleft()  # Get the node with lowest lower bound distance

            # Calculate distance from query point to node center
            dist_to_center = np.linalg.norm(point - node.center)

            # If the closest possible point in this ball is farther than our best,
            # we can skip this subtree
            if dist_to_center - node.radius > best_dist:
                continue

            if node.points is not None:
                # Leaf node - check all points
                distances = np.linalg.norm(node.points - point, axis=1)
                min_idx = np.argmin(distances)
                min_dist = distances[min_idx]

                if min_dist < best_dist:
                    best_dist = min_dist
                    best_point = node.points[min_idx]
            else:
                # Internal node - decide which child to visit first
                dist_to_p1 = np.linalg.norm(point - node.pivot1)
                dist_to_p2 = np.linalg.norm(point - node.pivot2)

                # Calculate lower bound distances to both subtrees
                # and visit them in order of potential closeness
                if dist_to_p1 <= dist_to_p2:
                    # Visit left subtree first, then right
                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        queue.appendleft((left_lb, node.left))  # Add to front to visit next

                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        if right_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((right_lb, node.right))
                else:
                    # Visit right subtree first, then left
                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        queue.appendleft((right_lb, node.right))  # Add to front to visit next

                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        if left_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((left_lb, node.left))

        return best_dist, best_point
#*-----------------------------------------------------------------------------

**Ball Tree Large Scale Examination**

In [14]:
import time
from tqdm import tqdm

# Test code for Ball Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 50_000  # Ensure inclusive range
leaf_size_step = 500

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the Ball Tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of Ball Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 50): 100%|██████████| 1000/1000 [00:00<00:00, 1698.32it/s]


-------------------------------------------------------------
Build Time: 6.5202 sec, Search Time: 0.5835 sec
Leaf Size: 50, Total Time: 7.1037 sec

2: === Testing with Leaf Size: 550, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 550): 100%|██████████| 1000/1000 [00:00<00:00, 1954.12it/s]


-------------------------------------------------------------
Build Time: 3.4309 sec, Search Time: 0.5065 sec
Leaf Size: 550, Total Time: 3.9374 sec

3: === Testing with Leaf Size: 1,050, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


...

-------------------------------------------------------------
Build Time: 1.4186 sec, Search Time: 3.4996 sec
Leaf Size: 49,050, Total Time: 4.9182 sec

100: === Testing with Leaf Size: 49,550, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 49,550): 100%|██████████| 1000/1000 [00:03<00:00, 288.16it/s]


-------------------------------------------------------------
Build Time: 1.4206 sec, Search Time: 3.4409 sec
Leaf Size: 49,550, Total Time: 4.8615 sec

101: === Testing with Leaf Size: 50,050, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 50,050): 100%|██████████| 1000/1000 [00:03<00:00, 277.93it/s]

-------------------------------------------------------------
Build Time: 1.3958 sec, Search Time: 3.5656 sec
Leaf Size: 50,050, Total Time: 4.9614 sec

=== Summary of Ball Tree Node Size Test ===
Leaf Size: 50, Build Time: 6.5202 sec, Search Time: 0.5835 sec, Total Time: 7.1037 sec
Leaf Size: 550, Build Time: 3.4309 sec, Search Time: 0.5065 sec, Total Time: 3.9374 sec
Leaf Size: 1050, Build Time: 3.0882 sec, Search Time: 0.5379 sec, Total Time: 3.6260 sec
Leaf Size: 1550, Build Time: 2.9109 sec, Search Time: 0.5965 sec, Total Time: 3.5074 sec
Leaf Size: 2050, Build Time: 2.7631 sec, Search Time: 0.6111 sec, Total Time: 3.3742 sec
Leaf Size: 2550, Build Time: 2.6545 sec, Search Time: 0.6771 sec, Total Time: 3.3316 sec
Leaf Size: 3050, Build Time: 2.5721 sec, Search Time: 0.7314 sec, Total Time: 3.3034 sec
Leaf Size: 3550, Build Time: 2.5232 sec, Search Time: 0.7833 sec, Total Time: 3.3065 sec
Leaf Size: 4050, Build Time: 2.4549 sec, Search Time: 0.8516 sec, Total Time: 3.3065 sec
Leaf 

**Ball Tree Large Scale (Run 2)**

In [18]:
import time
from tqdm import tqdm

# Test code for Ball Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 50_000  # Ensure inclusive range
leaf_size_step = 500

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the Ball Tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of Ball Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 50): 100%|██████████| 1000/1000 [00:00<00:00, 1814.00it/s]


-------------------------------------------------------------
Build Time: 6.4514 sec, Search Time: 0.5461 sec
Leaf Size: 50, Total Time: 6.9976 sec

2: === Testing with Leaf Size: 550, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 550): 100%|██████████| 1000/1000 [00:00<00:00, 1970.55it/s]


-------------------------------------------------------------
Build Time: 3.4781 sec, Search Time: 0.5031 sec
Leaf Size: 550, Total Time: 3.9812 sec

3: === Testing with Leaf Size: 1,050, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


...

-------------------------------------------------------------
Build Time: 1.4400 sec, Search Time: 3.5121 sec
Leaf Size: 49,050, Total Time: 4.9521 sec

100: === Testing with Leaf Size: 49,550, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 49,550): 100%|██████████| 1000/1000 [00:03<00:00, 281.62it/s]


-------------------------------------------------------------
Build Time: 1.4376 sec, Search Time: 3.5206 sec
Leaf Size: 49,550, Total Time: 4.9582 sec

101: === Testing with Leaf Size: 50,050, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 50,050): 100%|██████████| 1000/1000 [00:03<00:00, 283.99it/s]

-------------------------------------------------------------
Build Time: 1.4450 sec, Search Time: 3.4932 sec
Leaf Size: 50,050, Total Time: 4.9381 sec

=== Summary of Ball Tree Node Size Test ===
Leaf Size: 50, Build Time: 6.4514 sec, Search Time: 0.5461 sec, Total Time: 6.9976 sec
Leaf Size: 550, Build Time: 3.4781 sec, Search Time: 0.5031 sec, Total Time: 3.9812 sec
Leaf Size: 1050, Build Time: 3.1371 sec, Search Time: 0.5258 sec, Total Time: 3.6629 sec
Leaf Size: 1550, Build Time: 2.9240 sec, Search Time: 0.5657 sec, Total Time: 3.4897 sec
Leaf Size: 2050, Build Time: 2.7580 sec, Search Time: 0.6080 sec, Total Time: 3.3661 sec
Leaf Size: 2550, Build Time: 2.6908 sec, Search Time: 0.6572 sec, Total Time: 3.3480 sec
Leaf Size: 3050, Build Time: 2.5676 sec, Search Time: 0.7402 sec, Total Time: 3.3078 sec
Leaf Size: 3550, Build Time: 2.5223 sec, Search Time: 0.7925 sec, Total Time: 3.3147 sec
Leaf Size: 4050, Build Time: 2.4420 sec, Search Time: 0.8365 sec, Total Time: 3.2785 sec
Leaf 

**---------Ball Tree Optimum Node Size--------------**

In [19]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array([   50,  550, 1050, 1550, 2050, 2550, 3050, 3550, 4050, 4550, 5050, 5550,
  6050, 6550, 7050, 7550, 8050, 8550, 9050, 9550,10050,10550,11050,11550,
 12050,12550,13050,13550,14050,14550,15050,15550,16050,16550,17050,17550,
 18050,18550,19050,19550,20050,20550,21050,21550,22050,22550,23050,23550,
 24050,24550,25050,25550,26050,26550,27050,27550,28050,28550,29050,29550,
 30050,30550,31050,31550,32050,32550,33050,33550,34050,34550,35050,35550,
 36050,36550,37050,37550,38050,38550,39050,39550,40050,40550,41050,41550,
 42050,42550,43050,43550,44050,44550,45050,45550,46050,46550,47050,47550,
 48050,48550,49050,49550,50050])

# Total times from two runs (or versions)
total_times_1 = np.array([7.1037,3.9374,3.626 ,3.5074,3.3742,3.3316,3.3034,3.3065,3.3065,3.3326,
 3.325 ,3.3261,3.3329,3.3518,3.3676,3.3595,3.3963,3.3954,3.3913,3.3943,
 3.4387,3.3764,3.4007,3.377 ,3.4486,3.4581,3.6325,3.5037,3.5447,3.5237,
 3.5872,3.6467,3.5053,3.5501,3.567 ,3.5716,3.685 ,3.6947,3.6787,3.7201,
 3.6618,3.743 ,3.7066,3.737 ,3.7785,3.8147,3.7699,3.7588,3.7963,3.8468,
 3.8604,3.8377,3.7617,3.8413,3.8827,3.8864,3.9412,3.9595,3.9233,4.0198,
 4.0342,3.971 ,3.994 ,3.9922,4.0097,4.0111,4.0473,4.0696,4.0379,3.9838,
 4.0312,4.3307,4.317 ,4.3561,4.4282,4.3956,4.3664,4.3302,4.4355,4.4666,
 4.3898,4.4519,4.5199,4.472 ,4.4842,4.6335,4.671 ,4.6088,4.6356,4.852 ,
 4.8574,4.8324,4.8882,4.907 ,4.9189,4.9213,4.9681,4.8667,4.9182,4.8615,
 4.9614])

total_times_2 = np.array([6.9976,3.9812,3.6629,3.4897,3.3661,3.348 ,3.3078,3.3147,3.2785,3.284 ,
 3.3009,3.2961,3.3611,3.3352,3.3291,3.357 ,3.3583,3.3655,3.357 ,3.3628,
 3.3773,3.3891,3.4219,3.4187,3.4231,3.475 ,3.48  ,3.4751,3.5487,3.5735,
 3.5464,3.5442,3.5101,3.4954,3.574 ,3.5579,3.6154,3.6546,3.6973,3.7193,
 3.6944,3.7328,3.6856,3.71  ,3.7606,3.814 ,3.7398,3.7775,3.7322,3.8328,
 3.8914,3.8365,3.8509,3.9123,3.912 ,3.9582,3.9623,3.9867,3.9301,4.0174,
 4.0262,4.0246,4.0638,4.0395,4.0998,4.0985,4.0296,4.0764,3.9985,4.007 ,
 4.0673,4.2855,4.3722,4.2975,4.3381,4.3427,4.2588,4.3049,4.4346,4.4618,
 4.4584,4.4633,4.5072,4.5301,4.4578,4.6132,4.6116,4.5895,4.6586,4.7398,
 4.8449,4.7171,4.8623,4.8013,4.8953,4.894 ,4.8874,4.9414,4.9521,4.9582,
 4.9381])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 3.2925
Minimum found at leaf size: 4050


In [20]:
build_time_1 = np.array([6.5202,3.4309,3.0882,2.9109,2.7631,2.6545,2.5721,2.5232,2.4549,2.4368,
 2.3859,2.3651,2.3081,2.2626,2.2157,2.1835,2.1798,2.1736,2.1309,2.0943,
 2.0978,2.0655,2.0547,2.0237,2.0096,2.0126,2.012 ,2.0133,1.9483,1.954 ,
 1.9521,1.9352,1.8903,1.9102,1.9119,1.8803,1.874 ,1.8422,1.8249,1.8013,
 1.8011,1.8125,1.7803,1.824 ,1.7608,1.7642,1.7719,1.7269,1.7371,1.7221,
 1.72  ,1.7045,1.6789,1.6881,1.6636,1.674 ,1.6739,1.6871,1.6479,1.6899,
 1.6731,1.6473,1.6532,1.6677,1.6322,1.6325,1.6787,1.644 ,1.6334,1.6127,
 1.6241,1.5966,1.5717,1.5826,1.5972,1.6048,1.5693,1.5819,1.5356,1.5341,
 1.493 ,1.5291,1.5004,1.5261,1.5164,1.489 ,1.4814,1.4544,1.4888,1.4606,
 1.4524,1.4468,1.4405,1.4432,1.4185,1.4142,1.4456,1.4227,1.4186,1.4206,
 1.3958])

build_time_2 = np.array([6.4514,3.4781,3.1371,2.924 ,2.758 ,2.6908,2.5676,2.5223,2.442 ,2.3928,
 2.3816,2.3353,2.3348,2.2589,2.214 ,2.1884,2.1808,2.1667,2.1218,2.1161,
 2.0966,2.0767,2.099 ,2.0742,1.9942,2.0129,1.9962,1.9768,1.9437,1.9765,
 1.9621,1.9405,1.9217,1.892 ,1.9127,1.8637,1.863 ,1.836 ,1.8198,1.8247,
 1.8207,1.8251,1.8072,1.7917,1.7603,1.8004,1.7486,1.7445,1.7212,1.7134,
 1.7465,1.7075,1.7404,1.7235,1.6942,1.6901,1.7019,1.6868,1.6438,1.7063,
 1.6452,1.67  ,1.683 ,1.6678,1.6645,1.6828,1.6633,1.6553,1.6352,1.6305,
 1.6559,1.618 ,1.5759,1.5883,1.5997,1.6071,1.564 ,1.5592,1.5594,1.5519,
 1.5478,1.5204,1.5431,1.5242,1.5403,1.4907,1.4857,1.4728,1.5067,1.4548,
 1.469 ,1.4576,1.4444,1.4455,1.422 ,1.4464,1.4419,1.4475,1.44  ,1.4376,
 1.445 ])

avg_build_times = (build_time_1 + build_time_2) / 2

print(avg_build_times)

[6.4858  3.4545  3.11265 2.91745 2.76055 2.67265 2.56985 2.52275 2.44845
 2.4148  2.38375 2.3502  2.32145 2.26075 2.21485 2.18595 2.1803  2.17015
 2.12635 2.1052  2.0972  2.0711  2.07685 2.04895 2.0019  2.01275 2.0041
 1.99505 1.946   1.96525 1.9571  1.93785 1.906   1.9011  1.9123  1.872
 1.8685  1.8391  1.82235 1.813   1.8109  1.8188  1.79375 1.80785 1.76055
 1.7823  1.76025 1.7357  1.72915 1.71775 1.73325 1.706   1.70965 1.7058
 1.6789  1.68205 1.6879  1.68695 1.64585 1.6981  1.65915 1.65865 1.6681
 1.66775 1.64835 1.65765 1.671   1.64965 1.6343  1.6216  1.64    1.6073
 1.5738  1.58545 1.59845 1.60595 1.56665 1.57055 1.5475  1.543   1.5204
 1.52475 1.52175 1.52515 1.52835 1.48985 1.48355 1.4636  1.49775 1.4577
 1.4607  1.4522  1.44245 1.44435 1.42025 1.4303  1.44375 1.4351  1.4293
 1.4291  1.4204 ]


In [21]:
search_time_1 = np.array([0.5835,0.5065,0.5379,0.5965,0.6111,0.6771,0.7314,0.7833,0.8516,0.8958,
 0.9391,0.9609,1.0248,1.0891,1.1518,1.176 ,1.2165,1.2219,1.2604,1.3001,
 1.341 ,1.3109,1.346 ,1.3534,1.439 ,1.4455,1.6205,1.4904,1.5964,1.5696,
 1.6352,1.7114,1.615 ,1.6399,1.6551,1.6913,1.811 ,1.8525,1.8538,1.9188,
 1.8607,1.9305,1.9262,1.913 ,2.0177,2.0504,1.998 ,2.0319,2.0592,2.1247,
 2.1404,2.1332,2.0828,2.1532,2.2191,2.2124,2.2672,2.2724,2.2753,2.3299,
 2.3611,2.3237,2.3408,2.3245,2.3775,2.3785,2.3685,2.4256,2.4045,2.3711,
 2.4071,2.7341,2.7453,2.7735,2.8311,2.7908,2.7971,2.7484,2.8999,2.9325,
 2.8968,2.9228,3.0195,2.9459,2.9679,3.1445,3.1895,3.1544,3.1468,3.3913,
 3.405 ,3.3856,3.4477,3.4637,3.5003,3.5071,3.5225,3.444 ,3.4996,3.4409,
 3.5656])

search_time_2 = np.array([0.5461,0.5031,0.5258,0.5657,0.608 ,0.6572,0.7402,0.7925,0.8365,0.8912,
 0.9194,0.9609,1.0263,1.0763,1.1151,1.1686,1.1775,1.1989,1.2352,1.2467,
 1.2807,1.3124,1.3229,1.3445,1.4288,1.4621,1.4838,1.4983,1.605 ,1.5971,
 1.5843,1.6037,1.5884,1.6035,1.6613,1.6942,1.7524,1.8186,1.8775,1.8945,
 1.8737,1.9076,1.8783,1.9184,2.0003,2.0136,1.9913,2.033 ,2.011 ,2.1193,
 2.1449,2.129 ,2.1104,2.1888,2.2177,2.268 ,2.2604,2.2998,2.2863,2.311 ,
 2.381 ,2.3547,2.3808,2.3717,2.4353,2.4157,2.3662,2.4211,2.3633,2.3765,
 2.4114,2.6675,2.7964,2.7093,2.7385,2.7356,2.6948,2.7457,2.8752,2.91  ,
 2.9106,2.9429,2.964 ,3.0059,2.9175,3.1225,3.1259,3.1166,3.1519,3.285 ,
 3.3759,3.2595,3.4179,3.3557,3.4733,3.4476,3.4455,3.494 ,3.5121,3.5206,
 3.4932])

avg_search_times = (search_time_1 + search_time_2) / 2

print(avg_search_times)

[0.5648  0.5048  0.53185 0.5811  0.60955 0.66715 0.7358  0.7879  0.84405
 0.8935  0.92925 0.9609  1.02555 1.0827  1.13345 1.1723  1.197   1.2104
 1.2478  1.2734  1.31085 1.31165 1.33445 1.34895 1.4339  1.4538  1.55215
 1.49435 1.6007  1.58335 1.60975 1.65755 1.6017  1.6217  1.6582  1.69275
 1.7817  1.83555 1.86565 1.90665 1.8672  1.91905 1.90225 1.9157  2.009
 2.032   1.99465 2.03245 2.0351  2.122   2.14265 2.1311  2.0966  2.171
 2.2184  2.2402  2.2638  2.2861  2.2808  2.32045 2.37105 2.3392  2.3608
 2.3481  2.4064  2.3971  2.36735 2.42335 2.3839  2.3738  2.40925 2.7008
 2.77085 2.7414  2.7848  2.7632  2.74595 2.74705 2.88755 2.92125 2.9037
 2.93285 2.99175 2.9759  2.9427  3.1335  3.1577  3.1355  3.14935 3.33815
 3.39045 3.32255 3.4328  3.4097  3.4868  3.47735 3.484   3.469   3.50585
 3.48075 3.5294 ]
